# In this notebook I produce test cases for different lineshape to be used as tests in the C++ implementation

In [ ]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import AdvancedBraggEdgeFitting_v2
from scipy.stats import norm
from lmfit import Model
import lmfit

from TOF_routines import tof2l, l2tof, find_first, find_last, find_nearest, savitzky_golay
from AdvancedBraggEdgeFitting_v2 import running_mean
from scipy.optimize import curve_fit

%matplotlib notebook

## Load the test data, this is the main Bragg edge of the iron powder sample used for calibration of the IMAT experiment 

In [ ]:
x = np.loadtxt('../UnitTests/test_data/x.txt') # x axis, here in TOF
y = np.loadtxt('../UnitTests/test_data/y.txt') # y axis, here in transmission
l = tof2l(x,0,0,55) # this is fictional to produce the results 

plt.figure()
plt.plot(x,y)
plt.xlabel('TOF [s]')
plt.ylabel('Transmission [I/$I_{0}$]')

logy = -np.log(y)
plt.figure()
plt.plot(x,logy)
plt.xlabel('TOF [s]')
plt.ylabel('Attenuation -log(I/$I_{0}$)')
np.savetxt('../UnitTests/test_data/logy.txt',logy)

## First produced case in the transmission and exponential function (default) 
To be tested are:
1. initial edge shape computed with the initial parameters
2. fitted parameters

In [ ]:

# initial parameters (these are taken from the initial steps in AdvancedBraggEdgeFitting)
t0=0.056568
alpha=0.0015
sigma=0.0001
a1=0.315462
a2=5.3447841
a5=-0.4700811
a6=26.92982514130

mymodel_ini=AdvancedBraggEdgeFitting_v2.BraggEdgeExponential(x,t0,alpha,sigma,a1,a2,a5,a6,True)
myedge = AdvancedBraggEdgeFitting_v2.B(x,t0,alpha,sigma,True)

term3 = AdvancedBraggEdgeFitting_v2.term3(x,t0,sigma)
term4 = AdvancedBraggEdgeFitting_v2.term4(x,t0,alpha,sigma)
term5 = AdvancedBraggEdgeFitting_v2.term5(x,t0,alpha,sigma)

gmodel = Model(AdvancedBraggEdgeFitting_v2.BraggEdgeExponential)

# This by passes the implementation of BraggEdgeFitting which is iterative and makes some steps
params = gmodel.make_params(t0=t0,sigma=sigma, alpha=alpha, a1=a1, a2=a2, a5=a5, a6=a6)
params['bool_transmission'].vary = False


result = gmodel.fit(y, params, t=x, nan_policy='propagate')
print(result.fit_report())

plt.figure()
plt.plot(x,mymodel_ini) # Initial model
plt.plot(x,y) # Data
plt.plot(x, result.best_fit) # Best fit

# np.savetxt('../test_data/x.txt',x)
# np.savetxt('../test_data/y.txt',y)

## This possibly I don't need
# np.savetxt('../test_data/edge.txt', myedge) # This is just the edge shape, without the stuff before and after
# np.savetxt('../test_data/term3.txt', term3)
# np.savetxt('../test_data/term4.txt', term4)
# np.savetxt('../test_data/term5.txt', term5)
np.savetxt('../UnitTests/test_data/ini_model_Texp.txt', mymodel_ini)

## The second test case is Transmission and linear function before and after the edge 

In [ ]:
t0=0.056568
alpha=0.0015
sigma=0.0001

# I have to recompute the linear parameters, because the linear formulation is a bit different from the exponentials..
est_pos = find_nearest(t0,x)
x_before= x[0:est_pos]
bragg_before=y[0:est_pos]
x_after= x[est_pos+int(est_pos*0.2):-1]
bragg_after=y[est_pos+int(est_pos*0.2):-1]

# plt.figure()
# plt.plot(x_before, bragg_before)
# plt.plot(x_after, bragg_after)

[slope_before, interception_before] = np.polyfit(x_before, bragg_before, 1)
[slope_after, interception_after] = np.polyfit(x_after, bragg_after, 1)
#first guess of paramters
a2=slope_after
a5=interception_before
a6=slope_before
a1=interception_after

print(a1,a2,a5,a6)


mymodel_ini=AdvancedBraggEdgeFitting_v2.BraggEdgeLinear(x,t0,alpha,sigma,a1,a2,a5,a6,True)

np.savetxt('../UnitTests/test_data/ini_model_Tlin.txt', mymodel_ini)

plt.figure()
plt.plot(x,y)
plt.plot(x, mymodel_ini)

gmodel = Model(AdvancedBraggEdgeFitting_v2.BraggEdgeLinear)
params = gmodel.make_params(t0=t0,sigma=sigma, alpha=alpha, a1=a1, a2=a2, a5=a5, a6=a6, bool_transmission=True)
params['bool_transmission'].vary = False
result = gmodel.fit(y, params, t=x, nan_policy='propagate')
print(result.fit_report())
plt.plot(x, result.best_fit) # Best fit


## The third case is the gaussian of the gradient 
but first I will have to implement some sort of averaging in C++

In [ ]:
def gaussian(x, amp, cen, wid):
    return amp * np.exp(-(x-cen)**2 / wid)

In [ ]:
#Initial parameters
height = 500
t0=0.056568
sigma=0.0001

averaged = savitzky_golay(y, 31, 5)
# averaged = y
mygradient = np.gradient(averaged,x)
print(np.shape(mygradient), np.shape(y))

plt.figure()
plt.plot(l,y)
plt.xlabel('$\lambda$[$\AA$]', fontsize=14)
plt.ylabel('Transmission I/I$_{0}$', fontsize=14)
plt.title('Measured data', fontsize=16)
plt.savefig('meas_data.pdf')

plt.figure()
plt.plot(l,y)
plt.plot(l,averaged)
plt.xlabel('$\lambda$[$\AA$]', fontsize=14)
plt.ylabel('Transmission I/I$_{0}$', fontsize=14)
plt.title('Smoothed data', fontsize=16)
plt.savefig('smooth_data.pdf')


np.savetxt('../UnitTests/test_data/smoothed_edge.txt', averaged)

# %matplotlib notebook
plt.figure()
plt.plot(l,mygradient)
plt.xlabel('$\lambda$[$\AA$]', fontsize=14)
plt.ylabel('Gradient of transmission I/I$_{0}$', fontsize=14)
plt.title('Numerical gradient', fontsize=16)
plt.savefig('gradient_data.pdf')


np.savetxt('../UnitTests/test_data/gradient.txt', mygradient)

gmodel = Model(gaussian)
params = gmodel.make_params(cen=t0, amp=500, wid=0.0001)
# y_eval = gmodel.eval(params, x=x)

# plt.plot(x,y_eval)
ini_gauss = gaussian(x,height, t0, sigma)
np.savetxt('../UnitTests/test_data/ini_gauss.txt', ini_gauss)

best_vals, covar = curve_fit(gaussian, x, mygradient)
print(best_vals)
gaus_fit = gaussian(x,best_vals[0], best_vals[1], best_vals[2])

plt.figure()
ax = plt.subplot(111)
plt.plot(l,mygradient,alpha=0.8, label='numerical gradient')
plt.plot(l,gaus_fit, color='r',label='gaussian fit')
plt.plot(l, ini_gauss,color='g', alpha=0.9, label='first guess for fitting')
# computed ini_gauss
myini_gauss = height*np.exp(-(x-t0)**2/(sigma**2))
# plt.plot(x, myini_gauss)
                     
result = gmodel.fit(mygradient, params, x=x)
print(result.fit_report())

# plt.plot(l, result.best_fit) # Best fit

plt.xlabel('$\lambda$[$\AA$]', fontsize=14)
plt.ylabel('Gradient of transmission I/I$_{0}$', fontsize=14)
plt.title('Numerical gradient', fontsize=16)
lg1 = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.savefig('gaussfit_data.pdf',bbox_extra_artist=(lg1), bbox_inches='tight')

plt.figure()
plt.plot(l,y,label='measured data')
plt.plot(l,0.2+0.0005*mygradient,color='gray',alpha=0.9, label='numerical gradient')
plt.plot(l,0.2+0.0005*gaus_fit, color='r',alpha=0.6,label='gaussian fit')
plt.xlabel('$\lambda$[$\AA$]', fontsize=14)
plt.ylabel('Transmission I/I$_{0}$', fontsize=14)
plt.savefig('edge_gradient.pdf')



amp:  706.346498 +/- 8.90919586 (1.26%) (init = 500)
cen:  0.05793304 +/- 2.2146e-06 (0.00%) (init = 0.056568)
wid:  4.6231e-08 +/- 1.3469e-09 (2.91%) (init = 0.0001)

In [ ]:
b = 4.6231e-08
amp = A*0.5*np.sqrt(np.pi)*np.sqrt(b)
print(amp) # it is not correct..!


## The fourth case is the Bragg edge model in attenuation and exponential functions 
This I don't foresee it will be really used

In [ ]:
# This is actually to FIX!! in the actual formulation

t0=0.056568
alpha=0.0015
sigma=0.0001

est_pos = find_nearest(t0,x)
x_before= x[0:est_pos]
bragg_before=logy[0:est_pos]
x_after= x[est_pos+int(est_pos*0.4):-1]
bragg_after=logy[est_pos+int(est_pos*0.4):-1]

plt.figure()
plt.plot(x_before, bragg_before)
plt.plot(x_after, bragg_after)

[slope_before, interception_before] = np.polyfit(x_before, bragg_before, 1)
[slope_after, interception_after] = np.polyfit(x_after, bragg_after, 1)
#first guess of parameters
a2=slope_after
a5=interception_before
a6=slope_before
a1=interception_after

exp_model_before = Model(AdvancedBraggEdgeFitting_v2.exp_before)
params = exp_model_before.make_params(a5=a5, a6=a6)
result_exp_model_before = exp_model_before.fit(bragg_before,params,t=x_before)
a5=result_exp_model_before.best_values.get('a5')
a6=result_exp_model_before.best_values.get('a6')

exp_model_after = Model(AdvancedBraggEdgeFitting_v2.exp_combined)
params = exp_model_after.make_params(a1=a1, a2=a2, a5=a5, a6=a6)
params['a5'].vary = False
params['a6'].vary = False
result_exp_model_after = exp_model_after.fit(bragg_after,params,t=x_after)
a1=result_exp_model_after.best_values.get('a1')
a2=result_exp_model_after.best_values.get('a2')

print(a1,a2,a5,a6)





mymodel_ini=AdvancedBraggEdgeFitting_v2.BraggEdgeExponential_Attenuation(x,t0,alpha,sigma,a1,a2,a5,a6,False) ## This is very weird
myedge = AdvancedBraggEdgeFitting_v2.B(x,t0,alpha,sigma,False)


plt.figure()
plt.plot(x,myedge)
plt.plot(x,mymodel_ini)
plt.plot(x,logy)
plt.plot(x,interception_before+slope_before*x,'--r', label='fitted line before')
plt.plot(x,interception_after+slope_after*x,'--g', label='fitted line after')
plt.plot(x,AdvancedBraggEdgeFitting_v2.exp_before(x,a5,a6),'r', label='fitted exp after')
plt.plot(x,AdvancedBraggEdgeFitting_v2.exp_combined(x,a1,a2,a5,a6),'g', label='fitted exp combined')

gmodel = Model(AdvancedBraggEdgeFitting_v2.BraggEdgeExponential_Attenuation)

# This by passes the implementation of BraggEdgeFitting which is iterative and makes some steps
params = gmodel.make_params(t0=t0,sigma=sigma, alpha=alpha, a1=a1, a2=a2, a5=a5, a6=a6, bool_transmission=False)
params['bool_transmission'].vary = False

first_guess = gmodel.eval(params, t=x)


result = gmodel.fit(logy, params, t=x, nan_policy='propagate')
print(result.fit_report())

plt.figure()
plt.plot(x,first_guess) # Initial model
plt.plot(x,logy) # Data
plt.plot(x, result.best_fit) # Best fit

np.savetxt('../UnitTests/test_data/inimodel_Aexp.txt', mymodel_ini)

# The fifth case is the Bragg edge model in attenuation and linear functions


In [ ]:
t0=0.056568
alpha=0.0015
sigma=0.0001

est_pos = find_nearest(t0,x)
x_before= x[0:est_pos]
bragg_before=logy[0:est_pos]
x_after= x[est_pos+int(est_pos*0.2):-1]
bragg_after=logy[est_pos+int(est_pos*0.2):-1]

[slope_before, interception_before] = np.polyfit(x_before, bragg_before, 1)
[slope_after, interception_after] = np.polyfit(x_after, bragg_after, 1)
#first guess of paramters
# a2 = slope_after
# a5 = interception_before
# a6 = slope_before
# a1 = interception_after

a6 = slope_after
a1 = interception_before
a2 = slope_before
a5 = interception_after


print(a1,a2,a5,a6)

mymodel_ini=AdvancedBraggEdgeFitting_v2.BraggEdgeLinear(x,t0,alpha,sigma,a1,a2,a5,a6,False)
myedge = AdvancedBraggEdgeFitting_v2.B(x,t0,alpha,sigma,False)

plt.figure()
plt.plot(myedge)
plt.plot(mymodel_ini)

gmodel = Model(AdvancedBraggEdgeFitting_v2.BraggEdgeLinear)

# This by passes the implementation of BraggEdgeFitting which is iterative and makes some steps
params = gmodel.make_params(t0=t0,sigma=sigma, alpha=alpha, a1=a1, a2=a2, a5=a5, a6=a6, bool_tranmission='False')
params['bool_transmission'].vary = False


result = gmodel.fit(logy, params, t=x, nan_policy='propagate')
print(result.fit_report())

plt.figure()
plt.plot(x,mymodel_ini) # Initial model
plt.plot(x,logy) # Data
plt.plot(x, result.best_fit) # Best fit

np.savetxt('../UnitTests/test_data/inimodel_Alinear.txt', mymodel_ini)

## Here I produced the data that were used for testing the numerical gradient implementation 

In [ ]:

print(np.shape(x))
print(x[1::100])
print(y[1::100])
print(np.gradient(y[1::100], x[1::100]))

plt.figure()
plt.plot(x[1::100], y[1::100])

plt.figure()
plt.plot(x[1::100], np.gradient(y[1::100]))

## Here I test some method for edge height computation 

In [ ]:
plt.figure()
plt.plot(x, result.best_fit)
print(result.best_values.get('t0'))
edge_point = result.best_values.get('t0')
first_point = result.best_values.get('t0')-1.96*result.best_values.get('sigma')
second_point = result.best_values.get('t0')+1.96*result.best_values.get('sigma')+5*result.best_values.get('alpha')
plt.plot(first_point, result.best_fit[find_nearest(x,first_point)],'*')
plt.plot(second_point, result.best_fit[find_nearest(x,second_point)],'*')
plt.plot(edge_point, result.best_fit[find_nearest(x,edge_point)],'*')



print(result.best_values.get('t0'), result.best_values.get('sigma'), result.best_values.get('alpha'))

In [ ]:
plt.figure()
plt.plot(x, np.gradient(result.best_fit))
plt.plot(x, np.zeros(np.size(result.best_fit)))
plt.plot(x[np.gradient(result.best_fit)==0], (np.gradient(result.best_fit)[np.gradient(result.best_fit)==0]))

# Test cases for tof to lambda conversion


In [ ]:
mylambda = np.genfromtxt('../UnitTests/test_data/lambda.txt',usecols=0)
mytof = l2tof(mylambda,0,0,52)

relambda = tof2l(mytof,0,0,52)

np.savetxt('../UnitTests/test_data/tof.txt', mytof)

In [ ]:
plt.figure()
plt.plot(mytof, mylambda)

plt.figure()
plt.plot(mylambda, relambda)

# print(mylambda==relambda)
# print(mylambda)
# print(relambda)
print(len(mytof))

# Test for finding closest point to an array

In [ ]:
# print(mytof)
print(t0)
pos = find_nearest(mytof, t0)
print(mytof[pos])
print(pos)

# Test case for computing exponential functions